In [1]:
from pyspark.sql import SparkSession
from pyspark.sql import functions
from pyspark.sql.window import Window
from pyspark.sql.functions import col, row_number
from pyspark.sql.functions import split, regexp_replace
from pyspark.sql.functions import explode
from sparkmeasure import StageMetrics


spark = SparkSession.builder.appName('q10').config("spark.jars", "/home/athina/Downloads/spark-measure_2.12-0.17.jar").getOrCreate()
stagemetrics = StageMetrics(spark)
stagemetrics.begin()

df_mv = (spark.read.format('csv').option('header', 'true').option('infereSchema', 'true').load("movie.csv"))
df_tag = spark.read.format('csv').option('header', 'true').option('infereSchema', 'true').load('tag.csv')
df_rt = spark.read.format('csv').option('header', 'true').option('inferSchema', 'true').load('rating.csv')


#--Q10 START-------------------------------------------------------------------------------------------------------
gen = df_mv.select('movieId',split(df_mv.genres, '\|').alias('lgenres'))
#>>gen.show()

df_mv_plus = df_mv.join(gen, df_mv.movieId == gen.movieId, 'inner').drop(df_mv.movieId)


funny = df_tag.filter(df_tag["tag"].rlike('(?i)^(?!not|un).*funny.*$'))#.select('movieId').distinct()
#>>funny.orderBy('userId').show(10)

funny_with_ratings = funny.join(df_rt, funny.movieId == df_rt.movieId, 'inner').drop(funny.movieId)
#>>funny_with_ratings.show()

funny_with_above_3_5 = funny_with_ratings.filter(funny_with_ratings.rating > 3.5)
funny_with_above_3_5_same = funny_with_above_3_5.where(funny.userId == df_rt.userId)
#>>funny_with_above_3_5_same.show()

funny_with_above_3_5_same_mv = funny_with_above_3_5_same.join(df_mv, funny_with_above_3_5_same.movieId == df_mv.movieId, 'inner').drop(funny.timestamp).drop(df_rt.timestamp).drop(df_mv.movieId).drop(df_mv.genres)
#>>funny_with_above_3_5_same_mv.show()

funny_with_above_3_5_same_mv_list= funny_with_above_3_5_same_mv.join(df_mv_plus, funny_with_above_3_5_same_mv.movieId == df_mv_plus.movieId, 'inner').drop(funny.userId).drop(df_mv_plus.movieId).drop(df_mv_plus.title).drop(df_mv_plus.genres)
#>>funny_with_above_3_5_same_mv_list.show()

funny_with_above_3_5_same_mv_expl = funny_with_above_3_5_same_mv_list.withColumn("lgenres", explode(funny_with_above_3_5_same_mv_list.lgenres)).select('movieId', 'lgenres').distinct()
#>>funny_with_above_3_5_same_mv_expl.show()

#funny_with_above_3_5_same_mv_expl.groupBy('lgenres').count().show()#.agg(functions.count(func.lit(1)).alias("Num Of Records"))

#q10
funny_with_above_3_5_same_mv_expl.groupBy('lgenres').count().orderBy('lgenres').show(5)#.agg(functions.count(func.lit(1)).alias("Num Of Records"))

stagemetrics.end()

#print performance metrics
stagemetrics.print_report()
spark.stop()


#--Q10 END---------------------------------------------------------------------------------------------------------

+---------+-----+
|  lgenres|count|
+---------+-----+
|   Action|  125|
|Adventure|  127|
|Animation|   73|
| Children|   88|
|   Comedy|  540|
+---------+-----+
only showing top 5 rows


Scheduling mode = FIFO
Spark Context default degree of parallelism = 2
Aggregated Spark stage metrics:
numStages => 12
numTasks => 620
elapsedTime => 102951 (1.7 min)
stageDuration => 165078 (2.8 min)
executorRunTime => 167986 (2.8 min)
executorCpuTime => 136424 (2.3 min)
executorDeserializeTime => 6502 (7 s)
executorDeserializeCpuTime => 4227 (4 s)
resultSerializationTime => 134 (0.1 s)
jvmGCTime => 2910 (3 s)
shuffleFetchWaitTime => 0 (0 ms)
shuffleWriteTime => 4928 (5 s)
resultSize => 2591536 (2.0 MB)
diskBytesSpilled => 0 (0 Bytes)
memoryBytesSpilled => 0 (0 Bytes)
peakExecutionMemory => 6659047424
recordsRead => 30543075
bytesRead => 1407846935 (1342.0 MB)
recordsWritten => 0
bytesWritten => 0 (0 Bytes)
shuffleRecordsRead => 10027590
shuffleTotalBlocksFetched => 4741
shuffleLocalBlocksFetched => 